**Section 5b: Empirical Performance - 20 NewsGroups**

*Imports*

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.datasets import fetch_olivetti_faces
from collections import defaultdict
from random_matrix import *
from nmf import *
from benchmark import *

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import adjusted_rand_score
import numpy as np

*Data*

In [2]:
newsgroups = fetch_20newsgroups(
    subset='test',
    remove=('headers', 'footers', 'quotes')  # Keep only body text
)

texts = newsgroups.data 
true_labels = newsgroups.target

# Create TF-IDF vectorizer (limit to top 5000 terms)
tfidf = TfidfVectorizer(
    max_features=5000,        # Keep top 5k terms
    stop_words='english',    # Remove English stopwords
    min_df=5,                # Ignore terms in <5 docs
    max_df=0.7               # Ignore terms in >70% docs
)

# Apply vectorizer to texts -> Sparse matrix: (documents × terms)
X_tfidf = tfidf.fit_transform(texts)

In [3]:
e = np.ones(X_tfidf.shape[1])  # Vector of ones (shape: n_terms,)
term_weights = X_tfidf.T @ X_tfidf @ e  # Shape: (n_terms,)
term_weights = np.array(term_weights).flatten()  # Ensure 1D array

# 2. Avoid division by zero for unused terms
epsilon = 1e-6
D_inv_sqrt = np.diag(1 / np.sqrt(term_weights + epsilon))  # Sparse diagonal matrix

# 3. Apply weighting: X_ncw = X_tfidf D^{-1/2}
X_ncw = X_tfidf @ D_inv_sqrt

In [6]:
import numpy as np

def nmf_kl(A, r, max_iter=100, random_state=1, tol=1e-4):
    """
    Non-negative Matrix Factorization using KL-divergence with stable updates.

    Parameters:
        A (np.ndarray): Input non-negative matrix (m x n)
        r (int): Target rank
        max_iter (int): Maximum number of iterations
        tol (float): Convergence tolerance on reconstruction error
        random_state (int or None): Random seed

    Returns:
        W (np.ndarray): (m x r) basis matrix
        H (np.ndarray): (r x n) coefficient matrix
        errors (list): KL-divergence values during optimization
    """
    np.random.seed(random_state)
    m, n = A.shape
    
    # Initialize W, H with positive values
    W = np.abs(np.random.rand(m, r)) + 1e-16
    H = np.abs(np.random.rand(r, n)) + 1e-16
    
    errors = []
    for _ in range(max_iter):
        WH = W @ H
        
        # Update H (KL-divergence multiplicative update)
        numerator_H = W.T @ (A / (WH + 1e-16))
        denominator_H = W.T @ np.ones_like(A)
        H *= numerator_H / (denominator_H + 1e-16)
        
        # Update W (KL-divergence multiplicative update)
        WH = W @ H
        numerator_W = (A / (WH + 1e-16)) @ H.T
        denominator_W = np.ones_like(A) @ H.T
        W *= numerator_W / (denominator_W + 1e-16)
        
        # Compute KL-divergence (safely handle A=0)
        WH = W @ H
        kl_div = np.sum(np.where(A > 0, A * np.log(A / (WH + 1e-16)), 0) - A + WH)
        errors.append(kl_div)
        
        if kl_div < tol:
            break
            
    return W, H, errors

In [16]:
import numpy as np

def nmf_kl_compress_mu(A, r, max_iter=100, random_state=1, tol=1e-4, projection_type='gaussian'):
    """
    NMF with KL-divergence and randomized projections (compressed multiplicative updates).
    
    Parameters:
        A (np.ndarray): Input non-negative matrix (d x n)
        r (int): Target rank
        max_iter (int): Maximum number of iterations
        tol (float): Convergence tolerance on KL-divergence
        random_state (int): Random seed
        projection_type (str): Type of projection ('gaussian', 'sparse', etc.)
        
    Returns:
        F (np.ndarray): (d x r) basis matrix
        G (np.ndarray): (n x r) coefficient matrix (transposed for compatibility)
        errors (list): KL-divergence values during optimization
    """
    np.random.seed(random_state)
    d, n = A.shape
    eps = 1e-16  # Small constant for numerical stability

    # Generate projection matrices
    Rd = get_projection_matrix(projection_type, r, d)
    Rn = get_projection_matrix(projection_type, r, n)

    # Project input matrix
    Xd = Rd @ A  # Compressed view (r x n)
    Xn = A @ Rn.T  # Compressed view (d x r)

    # Initialize F and G with non-negative values
    F = np.abs(np.random.rand(d, r)) + eps
    G = np.abs(np.random.rand(n, r)) + eps

    errors = []
    for _ in range(max_iter):
        # --- Update G (n x r) ---
        F_tilde = Rd @ F  # Project F into compressed space (r x r)
        WH_compressed = F_tilde @ G.T  # Approx. of projected A (r x n)
        
        # KL-divergence update for G
        numerator_G = (Xd / (WH_compressed + eps)).T @ F_tilde  # (n x r)
        denominator_G = np.sum(F_tilde, axis=0, keepdims=True)  # (1 x r)
        G *= numerator_G / (denominator_G + eps)
        G = np.maximum(G, eps)

        # --- Update F (d x r) ---
        G_tilde = Rn @ G  # Project G into compressed space (r x r)
        WH_compressed = F @ G_tilde.T  # Approx. of projected A (d x r)
        
        # KL-divergence update for F
        numerator_F = (Xn / (WH_compressed + eps)) @ G_tilde  # (d x r)
        denominator_F = np.sum(G_tilde, axis=0, keepdims=True)  # (1 x r)
        F *= numerator_F / (denominator_F + eps)
        F = np.maximum(F, eps)

        # --- Compute KL-divergence ---
        WH = F @ G.T
        kl_div = np.sum(A * np.log(A / (WH + eps)) - A + WH)
        errors.append(kl_div)
        
        if kl_div < tol:
            break

    return F, G.T, errors

In [22]:
W,H,kl = nmf_kl_compress_mu(X_ncw,20,max_iter=100,projection_type='count-sketch')

C:\Users\Tom\AppData\Local\Temp\ipykernel_58164\1503739927.py:60: RuntimeWarning: divide by zero encountered in log
  kl_div = np.sum(A * np.log(A / (WH + eps)) - A + WH)
C:\Users\Tom\AppData\Local\Temp\ipykernel_58164\1503739927.py:60: RuntimeWarning: invalid value encountered in multiply
  kl_div = np.sum(A * np.log(A / (WH + eps)) - A + WH)


In [23]:
pred_labels = W.argmax(axis=1)
ari_score = adjusted_rand_score(true_labels, pred_labels)
ari_score

0.00017270104321436388

*Benchmark*

In [16]:
methods = {
    "MU C": nmf_compress_mu,
    'MU SC': nmf_structured_compress_mu,
    'HALS C': nmf_compress_hals,
    'HALS SC': nmf_structured_compress_hals
}

projection_types = [
    'gaussian',
    # 'srht',
    # 'givens',
     'srft',
     'sparse-jl',
     'count-sketch',
]

stats = {
'errors': {method: defaultdict(int) for method in methods},
'time': {method: defaultdict(int) for method in methods},
'ARI': {method: defaultdict(int) for method in methods}
}

# Set r
r = 20
runs = 1
rows = []
for method_name, method in methods.items():
    for projection in projection_types:
        total_errors = []    
        total_times = []  
        ari_scores = [] 
        for i in range(runs):
            # Set seed per run
            seed = i + 1
            
            # Time NMF Method
            start_time = timeit.default_timer()
            W, H, errors = method(X_ncw, r, random_state=seed,projection_type = projection)
            time = timeit.default_timer() - start_time

            # Get ARI Score
            pred_labels = W.argmax(axis=1)
            ari_score = adjusted_rand_score(true_labels, pred_labels)
            # Store
            total_times.append(time)
            total_errors.append(errors[-1])
            ari_scores.append(ari_score)
        
        # Store average times
        rows.append({
            'Method':method_name,
            'Projection':projection,
            'Time':np.mean(total_times),
            'Errors':np.mean(total_errors),
            'ARI':np.mean(ari_scores)
        })
        print(f"Completed {method_name},{projection}")

Completed MU C,gaussian
Completed MU C,srft
Completed MU C,sparse-jl
Completed MU C,count-sketch
Completed MU SC,gaussian
Completed MU SC,srft
Completed MU SC,sparse-jl
Completed MU SC,count-sketch
Completed HALS C,gaussian
Completed HALS C,srft
Completed HALS C,sparse-jl
Completed HALS C,count-sketch
Completed HALS SC,gaussian
Completed HALS SC,srft
Completed HALS SC,sparse-jl
Completed HALS SC,count-sketch


In [17]:
pd.DataFrame(rows)

,Method,Projection,Time,Errors,ARI
0,MU C,gaussian,11.931203,1.076159,0.001140
1,MU C,srft,11.861955,0.997384,0.001270
2,MU C,sparse-jl,12.121970,1.111360,0.000797
3,MU C,count-sketch,11.935314,1.077865,0.000916
4,MU SC,gaussian,12.076636,0.986291,0.197624
5,MU SC,srft,24.759421,0.986083,0.154895
6,MU SC,sparse-jl,12.163921,0.987753,0.195202
7,MU SC,count-sketch,12.126181,0.989150,0.188550
8,HALS C,gaussian,11.606488,1.009426,0.000605
9,HALS C,srft,11.618306,0.998528,0.000892


*Plots*